# Урок 5. Рекуррентные нейронные сети

## Практическое задание

1. Попробуйте обучить нейронную сеть LSTM на любом другом датасете. Опишите в комментарии к уроку - какой результата вы добились от нейросети? Что помогло вам улучшить ее точность?
2. *Попробуйте на numpy реализовать нейронную сеть архитектуры LSTM
3. *Предложите свои варианты решения проблемы исчезающего градиента в RNN

### 1. LSTM для IMDB


In [1]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.regularizers import L1L2
from keras.optimizers import SGD, RMSprop, Adam

max_features = 20000
maxlen = 80

print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

print('Pad последовательности (примеров в x единицу времени)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности
Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [2]:
print('Построение модели...')
model = Sequential()

model.add(Embedding(max_features, 128))
model.add(LSTM(
    64, 
    dropout=0.2, 
    bias_regularizer=L1L2(0.01, 0.01)))  # L1L2(0, 0.01)

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',  # SGD(learning_rate=0.01, momentum=0.9),
              metrics=['accuracy'])

print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=16,
          epochs=1,
          validation_data=(x_test, y_test))

Построение модели...
Процесс обучения...
1563/1563 [==============================] - 76s 49ms/step - loss: 0.8557 - accuracy: 0.8019 - val_loss: 0.3846 - val_accuracy: 0.8417


Модель немного улучшилась, благодаря добавлению двойной регуляризации, так как, заметил, что модель сильно переобучалась.
Также пробовал менять коэффициент Dropout, что не привело к улучшению модели.
Аналогично другие оптимизаторы не привели к улучшению. Лучше работает Adam с дефолтными парамтерами в данной ситуации. 

### 2. Генерация текста на основе книги Алисы в стране чудес

In [3]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential


# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)


# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    LSTM(
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        dropout=0.1, 
        bias_regularizer=L1L2(0.01, 0.01),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(
        X, 
        y, 
        batch_size=BATCH_SIZE, 
        epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Итерация #: 0
1241/1241 [==============================] - 12s 10ms/step - loss: 3.3531
Генерация из посева: f the proj
f the projent the sout the sout the sout the sout the sout the sout the sout the sout the sout the sout the so==================================================
Итерация #: 1
1241/1241 [==============================] - 12s 10ms/step - loss: 2.1363
Генерация из посева: his time, 
his time, and the groped the said the wast the groped the said the wast the groped the said the wast the grope==================================================
Итерация #: 2
1241/1241 [==============================] - 12s 10ms/step - loss: 1.9709
Генерация из посева:  ive read 
 ive read the dore the dore the dore the dore the dore the dore the dore the dore the dore the dore the dore t==================================================
Итерация #: 3
1241/1241 [==============================] - 12s 10ms/step - loss: 1.8649
Генерация из посева: ear! oh de
ear! oh der and the mork to the was h

LSTM показала лучший результат. Также регуляризация и Dropout немного улучшили обучение. 